In [16]:
import pandas as pd
import plotly.express as px
from datetime import timedelta
import plotly.graph_objects as go
from pyod.models.knn import KNN
from pyod.models.ecod import ECOD
from scipy.stats import chi2_contingency


#### Importieren der Datei

In [3]:
# Laden der Excel-Datei in ein Pandas Dataframe
df_psp = pd.read_excel('C:/Users/Ibrom/Studium/DLMDWME01/PSP_Jan_Feb_2019.xlsx', sheet_name='Sheet1')

# Umbenennen der ersten Spalte in 'Id'
df_psp = df_psp.rename(columns={df_psp.columns[0]: 'Id'})

df_psp.head()

,Id,tmsp,country,amount,success,PSP,3D_secured,card
0,0,2019-01-01 00:01:11,Germany,89,0,UK_Card,0,Visa
1,1,2019-01-01 00:01:17,Germany,89,1,UK_Card,0,Visa
2,2,2019-01-01 00:02:49,Germany,238,0,UK_Card,1,Diners
3,3,2019-01-01 00:03:13,Germany,238,1,UK_Card,1,Diners
4,4,2019-01-01 00:04:33,Austria,124,0,Simplecard,0,Diners


In [ ]:
# Add 57 month to transfer date from Jan-Feb 2019 to Okt-Nov 2023
df_psp.tmsp = df_psp.tmsp + pd.DateOffset(months=57)

#### Allgemeine Informationen zum Datensatz

In [ ]:
# Anzahl der Spalten
num_cols = len(df_psp.columns)
print(f"Die Anzahl der Spalten ist: {num_cols}")

In [ ]:
# Namen der Spalten
col_names = df_psp.columns.tolist()
print(f"Die Namen der Spalten sind: {col_names}")

In [ ]:
# Datentypen der Spalten
col_types = df_psp.dtypes
print(f"Die Datentypen der Spalten sind: \n {col_types}")

In [ ]:
# Mindest und Maximalwert der Spalten mit dem Namen 
# (angenommen, es bezieht sich auf die numerischen Spalten)
min_max_values = df_psp.describe().loc[['min','max']]
print(f"Mindest- und Maximalwerte der numerischen Spalten sind: \n {min_max_values}")

In [ ]:
# Anzahl der Zeilen
num_rows = len(df_psp)
print(f"Die Anzahl der Zeilen ist: {num_rows}")

#### Untersuchen des Merkmals "tmsp"

In [ ]:
# Erstellen Sie eine Kopie von df_psp und konvertieren Sie die 'tmsp' Spalte zu datetime
df_psp_tmsp_date = df_psp.copy()
df_psp_tmsp_date['tmsp'] = pd.to_datetime(df_psp_tmsp_date['tmsp'])

In [ ]:
# Erstelle einen neuen DataFrame für 'tmsp'
df_tmsp = df_psp_tmsp_date.groupby('tmsp')['Id'].nunique().reset_index()
df_tmsp.columns = ['tmsp', 'count']

# Erstelle das Balkendiagramm
fig = px.bar(df_tmsp, x="tmsp", y="count", title="Balkendiagramm für 'tmsp'")
fig.show()

In [ ]:
# Konvertieren Sie die 'tmsp' Spalte zu datetime, falls noch nicht geschehen
df_psp['tmsp'] = pd.to_datetime(df_psp['tmsp'])

# Erstellen Sie einen vollständigen Datumsbereich für den gegebenen Zeitraum
date_range = pd.date_range(start='2023-10-01', end='2023-11-28')

# Finden Sie die fehlenden Daten heraus
missing_dates = date_range.difference(df_psp['tmsp'].dt.normalize()).tolist()

print("Fehlende Daten:", missing_dates)

In [ ]:

# Stelle sicher, dass 'tmsp' eine datetime Spalte ist
df_psp['tmsp'] = pd.to_datetime(df_psp['tmsp'])

# Sortiere df_psp nach 'tmsp'
df_psp_sorted = df_psp.sort_values(by='tmsp')

# Berechne den zeitlichen Abstand zwischen aufeinanderfolgenden Einträgen
df_psp_sorted['time_diff'] = df_psp_sorted['tmsp'].diff().dt.total_seconds()

# Entferne NaN-Werte und konvertiere den Abstand in Sekunden zu einem timedelta
df_psp_sorted['time_diff_timedelta'] = df_psp_sorted['time_diff'].dropna().apply(lambda x: timedelta(seconds=int(x)))

# Entferne die erste Zeile, die NaN-Werte enthält
df_psp_sorted = df_psp_sorted.iloc[1:, :]

# Erstelle ein neues DataFrame mit den gewünschten Spalten
df_time_diffs = pd.DataFrame({
    "Erste Zeile": df_psp_sorted.index[:-1],
    "Zweite Zeile": df_psp_sorted.index[1:],
    "tmsp erste Zeile": df_psp_sorted['tmsp'].values[:-1],
    "tmsp zweite Zeile": df_psp_sorted['tmsp'].values[1:],
    "Abstand [Sekunden]": df_psp_sorted['time_diff'].values[1:],
    "Abstand [Time]": df_psp_sorted['time_diff_timedelta'].values[1:]
})

# Sortiere nach "Abstand [Sekunden]" absteigend
df_time_diffs = df_time_diffs.sort_values(by="Abstand [Sekunden]", ascending=False)

df_time_diffs.head(5)

In [ ]:
# Berechne den Mittelwert
mean = df_time_diffs['Abstand [Sekunden]'].mean()

# Berechne die Standardabweichung
std_dev = df_time_diffs['Abstand [Sekunden]'].std()

# Berechne die Varianz
variance = df_time_diffs['Abstand [Sekunden]'].var()

print(f"Mittelwert: {mean}, Standardabweichung: {std_dev}, Varianz: {variance}")

#### Untersuchen des Merkmals "country"

In [ ]:
# Einzigartige Werte in 'country'
unique_countries = df_psp['country'].unique()

print("Einzigartige Länder:", unique_countries)

In [ ]:
# Erstelle einen neuen DataFrame für 'country'
df_country = df_psp_tmsp_date.groupby('country')['Id'].nunique().reset_index()
df_country.columns = ['country', 'count']

# Erstelle das Balkendiagramm
fig = px.bar(df_country, x="country", y="count", title="Balkendiagramm für 'country'")
fig.show()

In [ ]:
# Erstellen Sie eine Kopie von df_psp und konvertieren Sie die 'tmsp' Spalte zu datetime
df_psp_time = df_psp.copy()
df_psp_time['tmsp'] = pd.to_datetime(df_psp_time['tmsp'])

# Abrunden der 'tmsp' Werte auf die nächstgelegenen 15-Minuten-Intervalle
df_psp_time['time'] = df_psp_time['tmsp'].dt.round('15min').dt.time

# Ändern Sie 'success' in einen kategorischen Typ
df_psp_time['country'] = df_psp_time['country'].astype('category')

# Gruppieren Sie nach 'time' und 'success', und zählen Sie die einzigartige Anzahl von 'Id'
df_grouped = df_psp_time.groupby(['time', 'country'])['Id'].nunique().reset_index()
df_grouped.columns = ['time', 'country', 'count']

# Erstellen Sie das Balkendiagramm
fig = px.bar(df_grouped, x='time', y='count', color='country', title="Balkendiagramm für 'country'")
fig.update_layout(barmode='group')
fig.show()

#### Untersuchen des Merkmals "amount"

In [ ]:
# Erstelle einen neuen DataFrame für 'amount'
df_amount = df_psp_tmsp_date.groupby('amount')['Id'].nunique().reset_index()
df_amount.columns = ['amount', 'count']

# Erstelle das Balkendiagramm
fig = px.bar(df_amount, x="amount", y="count", title="Balkendiagramm für 'amount'")
fig.show()

#### Untersuchen des Merkmals "success"

In [ ]:
# Erstelle einen neuen DataFrame für 'success'
df_success = df_psp_tmsp_date.groupby('success')['Id'].nunique().reset_index()
df_success.columns = ['success', 'count']

# Erstelle das Balkendiagramm
fig = px.bar(df_success, x="success", y="count", title="Balkendiagramm für 'success'")
fig.show()

In [ ]:
# Erstellen Sie eine Kopie von df_psp und konvertieren Sie die 'tmsp' Spalte zu datetime
df_psp_time = df_psp.copy()
df_psp_time['tmsp'] = pd.to_datetime(df_psp_time['tmsp'])

# Abrunden der 'tmsp' Werte auf die nächstgelegenen 15-Minuten-Intervalle
df_psp_time['time'] = df_psp_time['tmsp'].dt.round('15min').dt.time

# Ändern Sie 'success' in einen kategorischen Typ
df_psp_time['success'] = df_psp_time['success'].astype('category')

# Gruppieren Sie nach 'time' und 'success', und zählen Sie die einzigartige Anzahl von 'Id'
df_grouped = df_psp_time.groupby(['time', 'success'])['Id'].nunique().reset_index()
df_grouped.columns = ['time', 'success', 'count']

# Erstellen Sie das Balkendiagramm
fig = px.bar(df_grouped, x='time', y='count', color='success', title="Balkendiagramm für 'success'")
fig.update_layout(barmode='group')
fig.show()

In [ ]:
# Erstellen Sie eine Kopie von df_psp und konvertieren Sie die 'tmsp' Spalte zu datetime
df_psp_time = df_psp.copy()
df_psp_time['tmsp'] = pd.to_datetime(df_psp_time['tmsp'])

# Abrunden der 'tmsp' Werte auf die nächstgelegenen 15-Minuten-Intervalle
df_psp_time['time'] = df_psp_time['tmsp'].dt.round('15min').dt.time

# Ändern Sie 'success' in einen kategorischen Typ
df_psp_time['success'] = df_psp_time['success'].astype('category')

countries = ["Germany", "Austria", "Switzerland"]

for country in countries:
    # Filtern Sie den DataFrame nach dem Land
    df_country = df_psp_time[df_psp_time['country'] == country]

    # Gruppieren Sie nach 'time' und 'success', und zählen Sie die einzigartige Anzahl von 'Id'
    df_grouped = df_country.groupby(['time', 'success'])['Id'].nunique().reset_index()
    df_grouped.columns = ['time', 'success', 'count']

    # Erstellen Sie das Balkendiagramm
    fig = px.bar(df_grouped, x='time', y='count', color='success', title=f"Balkendiagramm für 'success' in {country}")
    fig.update_layout(barmode='group')
    fig.show()

In [ ]:
# Erstellen Sie eine Kopie von df_psp und konvertieren Sie die 'tmsp' Spalte zu datetime
df_psp_time = df_psp.copy()
df_psp_time['tmsp'] = pd.to_datetime(df_psp_time['tmsp'])

# Abrunden der 'tmsp' Werte auf die nächstgelegenen 15-Minuten-Intervalle
df_psp_time['time'] = df_psp_time['tmsp'].dt.round('15min').dt.time

# Ändern Sie 'success' in einen kategorischen Typ
df_psp_time['success'] = df_psp_time['success'].astype('category')

PSPs = ["UK_Card", "Simplecard", "Moneycard", "Goldcard"]

for PSP in PSPs:
    # Filtern Sie den DataFrame nach dem Land
    df_PSP = df_psp_time[df_psp_time['PSP'] == PSP]

    # Gruppieren Sie nach 'time' und 'success', und zählen Sie die einzigartige Anzahl von 'Id'
    df_grouped = df_PSP.groupby(['time', 'success'])['Id'].nunique().reset_index()
    df_grouped.columns = ['time', 'success', 'count']

    # Erstellen Sie das Balkendiagramm
    fig = px.bar(df_grouped, x='time', y='count', color='success', title=f"Balkendiagramm für 'success' in {PSP}")
    fig.update_layout(barmode='group')
    fig.show()

#### Untersuchen des Merkmals "PSP"

In [ ]:
# Einzigartige Werte in 'country'
unique_PSP = df_psp['PSP'].unique()

print("Einzigartige PSP:", unique_PSP)

In [ ]:
# Erstelle einen neuen DataFrame für 'PSP'
df_PSP = df_psp_tmsp_date.groupby('PSP')['Id'].nunique().reset_index()
df_PSP.columns = ['PSP', 'count']

# Erstelle das Balkendiagramm
fig = px.bar(df_PSP, x="PSP", y="count", title="Balkendiagramm für 'PSP'")
fig.show()

#### Untersuchen des Merkmals "3D_secured"

In [ ]:
# Erstelle einen neuen DataFrame für '3D_secured'
df_3D_secured = df_psp_tmsp_date.groupby('3D_secured')['Id'].nunique().reset_index()
df_3D_secured.columns = ['3D_secured', 'count']

# Erstelle das Balkendiagramm
fig = px.bar(df_3D_secured, x="3D_secured", y="count", title="Balkendiagramm für '3D_secured'")
fig.show()

#### Untersuchen des Merkmals "card"

In [ ]:
# Erstelle einen neuen DataFrame für '3D_secured'
df_card = df_psp_tmsp_date.groupby('card')['Id'].nunique().reset_index()
df_card.columns = ['card', 'count']

# Erstelle das Balkendiagramm
fig = px.bar(df_card, x="card", y="count", title="Balkendiagramm für 'card'")
fig.show()

### Ausreißer Analyse mit PyOD

In [4]:
# Konvertiere 'tmsp' in Sekunden seit der Unix-Epoche
#df_psp['tmsp_seconds'] = (df_psp['tmsp'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
#df_psp.head()

,Id,tmsp,country,amount,success,PSP,3D_secured,card,tmsp_seconds
0,0,2019-01-01 00:01:11,Germany,89,0,UK_Card,0,Visa,1546300871
1,1,2019-01-01 00:01:17,Germany,89,1,UK_Card,0,Visa,1546300877
2,2,2019-01-01 00:02:49,Germany,238,0,UK_Card,1,Diners,1546300969
3,3,2019-01-01 00:03:13,Germany,238,1,UK_Card,1,Diners,1546300993
4,4,2019-01-01 00:04:33,Austria,124,0,Simplecard,0,Diners,1546301073


In [5]:

# Wählen Sie die numerischen Spalten aus
#df_psp_numerical_ECOD = df_psp[["Id", "amount", "success"]]
df_psp_numerical_ECOD = df_psp[["tmsp_seconds"]]

# Instanziieren Sie das ECOF-Modell
clf = ECOD(contamination=0.01)  # setze den Anteil der Ausreißer auf 1%

# Führe die Modellanpassung aus
clf.fit(df_psp_numerical_ECOD)

# Berechne die Ausreißerwahrscheinlichkeit für jeden Datenpunkt
scores_pred = clf.decision_function(df_psp_numerical_ECOD)

# Klassifiziere die Datenpunkte als inliers (1) oder outliers (-1)
y_pred = clf.predict(df_psp_numerical_ECOD)

# Erstelle einen neuen DataFrame und füge die Ausreißerwahrscheinlichkeit und die Klassifikation hinzu
df_psp_outliers_ECOD = df_psp.copy()
df_psp_outliers_ECOD['outlier_score'] = scores_pred
df_psp_outliers_ECOD['outlier'] = y_pred
df_psp_outliers_ECOD.head()

,Id,tmsp,country,amount,success,PSP,3D_secured,card,tmsp_seconds,outlier_score,outlier
0,0,2019-01-01 00:01:11,Germany,89,0,UK_Card,0,Visa,1546300871,10.827945,1
1,1,2019-01-01 00:01:17,Germany,89,1,UK_Card,0,Visa,1546300877,10.134798,1
2,2,2019-01-01 00:02:49,Germany,238,0,UK_Card,1,Diners,1546300969,9.729333,1
3,3,2019-01-01 00:03:13,Germany,238,1,UK_Card,1,Diners,1546300993,9.441650,1
4,4,2019-01-01 00:04:33,Austria,124,0,Simplecard,0,Diners,1546301073,9.218507,1


In [6]:
df_psp_outliers_ECOD.sort_values("outlier_score", ascending=False)

,Id,tmsp,country,amount,success,PSP,3D_secured,card,tmsp_seconds,outlier_score,outlier
0,0,2019-01-01 00:01:11,Germany,89,0,UK_Card,0,Visa,1546300871,10.827945,1
50409,50409,2019-02-28 23:48:19,Austria,91,1,Moneycard,0,Master,1551397699,10.827945,1
50408,50408,2019-02-28 23:47:36,Austria,91,0,UK_Card,0,Master,1551397656,10.134798,1
1,1,2019-01-01 00:01:17,Germany,89,1,UK_Card,0,Visa,1546300877,10.134798,1
2,2,2019-01-01 00:02:49,Germany,238,0,UK_Card,1,Diners,1546300969,9.729333,1
...,...,...,...,...,...,...,...,...,...,...,...
25207,25207,2019-01-30 15:27:33,Germany,199,1,Moneycard,0,Master,1548862053,0.693227,0
25206,25206,2019-01-30 15:24:14,Austria,44,1,Moneycard,1,Master,1548861854,0.693187,0
25203,25203,2019-01-30 15:17:32,Austria,95,1,UK_Card,0,Visa,1548861452,0.693187,0
25204,25204,2019-01-30 15:19:10,Austria,165,0,Moneycard,0,Visa,1548861550,0.693147,0


In [ ]:
# Wählen Sie die numerischen Spalten aus
#df_psp_numerical_KNN = df_psp[["Id", "amount", "success"]]
df_psp_numerical_KNN = df_psp[["amount"]]

# Instanziieren Sie das KNN-Modell
clf = KNN(contamination=0.01)  # setze den Anteil der Ausreißer auf 1%

# Führe die Modellanpassung aus
clf.fit(df_psp_numerical_KNN)

# Berechne die Ausreißerwahrscheinlichkeit für jeden Datenpunkt
scores_pred = clf.decision_function(df_psp_numerical_KNN)

# Klassifiziere die Datenpunkte als inliers (1) oder outliers (-1)
y_pred = clf.predict(df_psp_numerical_KNN)

# Erstelle einen neuen DataFrame und füge die Ausreißerwahrscheinlichkeit und die Klassifikation hinzu
df_psp_outliers_KNN = df_psp.copy()
df_psp_outliers_KNN['outlier_score'] = scores_pred
df_psp_outliers_KNN['outlier'] = y_pred

In [ ]:
df_psp_outliers_KNN.head()

In [ ]:
df_psp_outliers_KNN.sort_values("outlier_score", ascending=False).head(55)

In [ ]:

# Erstelle ein Histogramm der outlier_score-Werte
fig = px.histogram(df_psp_outliers_KNN, x="outlier_score", title="Histogramm der outlier_score-Werte")
fig.show()

# Erstelle ein Streudiagramm von 'amount' gegen 'outlier_score'
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_psp_outliers_KNN['amount'],
    y=df_psp_outliers_KNN['outlier_score'],
    mode='markers',
    marker=dict(
        color=df_psp_outliers_KNN['outlier'],
        colorscale='Viridis',
        opacity=0.8,
        size=5
    ),
    text=df_psp_outliers_KNN.index,
))
fig.update_layout(title='Streudiagramm von amount gegen outlier_score', 
                  xaxis_title='amount', yaxis_title='outlier_score')
fig.show()

In [ ]:
#Alles über 541 abschneiden

In [21]:

# Erstellen Sie eine Kreuztabelle der Variablen 'card' und 'PSP'
cross_tab = pd.crosstab(df_psp['card'], df_psp['PSP'])

print(cross_tab)

# Führen Sie den Chi-Quadrat-Test durch
chi2, p, dof, expected = chi2_contingency(cross_tab)

print(f"\nChi-Quadrat Statistik: {chi2}")
print(f"p-Wert: {p}")

PSP     Goldcard  Moneycard  Simplecard  UK_Card
card                                            
Diners       670       1667        2356     5075
Master      1763       4765        7245    15229
Visa         775       1865        2845     6155

Chi-Quadrat Statistik: 16.04005658272315
p-Wert: 0.013540575127843202


In [20]:

# Erstellen Sie eine Kreuztabelle der Variablen 'Success' und 'PSP'
cross_tab = pd.crosstab(df_psp['success'], df_psp['PSP'])

print(cross_tab)

# Führen Sie den Chi-Quadrat-Test durch
chi2, p, dof, expected = chi2_contingency(cross_tab)

print(f"\nChi-Quadrat Statistik: {chi2}")
print(f"p-Wert: {p}")

PSP      Goldcard  Moneycard  Simplecard  UK_Card
success                                          
0            1905       6482       10478    21317
1            1303       1815        1968     5142

Chi-Quadrat Statistik: 998.7783848081615
p-Wert: 3.312365253382584e-216


### Korrelationsanalyse

In [29]:
df_encoded_psp = pd.get_dummies(df_psp, columns=['PSP'])
df_encoded_psp.head()
correlation_matrix_psp = df_encoded_psp[['amount','PSP_Goldcard', 'PSP_Moneycard', 'PSP_Simplecard', 'PSP_UK_Card']].corr()
print(correlation_matrix_psp)

                  amount  PSP_Goldcard  PSP_Moneycard  PSP_Simplecard  \
amount          1.000000      0.002707      -0.002318       -0.006180   
PSP_Goldcard    0.002707      1.000000      -0.115715       -0.149268   
PSP_Moneycard  -0.002318     -0.115715       1.000000       -0.254145   
PSP_Simplecard -0.006180     -0.149268      -0.254145        1.000000   
PSP_UK_Card     0.005734     -0.274007      -0.466528       -0.601802   

                PSP_UK_Card  
amount             0.005734  
PSP_Goldcard      -0.274007  
PSP_Moneycard     -0.466528  
PSP_Simplecard    -0.601802  
PSP_UK_Card        1.000000  


In [42]:
df_encoded_card = pd.get_dummies(df_psp, columns=['card'])
#df_encoded_card.head()
correlation_matrix_card = df_encoded_card[['amount','card_Diners', 'card_Master', 'card_Visa']].corr()
print(correlation_matrix_card)

               amount  card_Diners  card_Master  card_Visa
amount       1.000000     0.000303    -0.002547   0.002703
card_Diners  0.000303     1.000000    -0.570612  -0.268623
card_Master -0.002547    -0.570612     1.000000  -0.637756
card_Visa    0.002703    -0.268623    -0.637756   1.000000


In [41]:
df_encoded_card_success = pd.get_dummies(df_psp, columns=['card'])
df_encoded_card_success = pd.get_dummies(df_encoded_card_success, columns=['success'])
df_encoded_card_success.head()
correlation_matrix_card_success = df_encoded_card_success[['success_0','success_1', 'card_Diners', 'card_Master', 'card_Visa']].corr()
print(correlation_matrix_card_success)

             success_0  success_1  card_Diners  card_Master  card_Visa
success_0     1.000000  -1.000000    -0.024474     0.026086  -0.007642
success_1    -1.000000   1.000000     0.024474    -0.026086   0.007642
card_Diners  -0.024474   0.024474     1.000000    -0.570612  -0.268623
card_Master   0.026086  -0.026086    -0.570612     1.000000  -0.637756
card_Visa    -0.007642   0.007642    -0.268623    -0.637756   1.000000


In [46]:
df_encoded_psp_success = pd.get_dummies(df_psp, columns=['PSP'])
df_encoded_psp_success = pd.get_dummies(df_encoded_psp_success, columns=['success'])
df_encoded_psp_success.head()
correlation_matrix_psp_success = df_encoded_psp_success[['success_0','success_1', 'PSP_Goldcard', 'PSP_Moneycard', 'PSP_Simplecard', 'PSP_UK_Card']].corr()
print(correlation_matrix_psp_success)

                success_0  success_1  PSP_Goldcard  PSP_Moneycard  \
success_0        1.000000  -1.000000     -0.131774      -0.017502   
success_1       -1.000000   1.000000      0.131774       0.017502   
PSP_Goldcard    -0.131774   0.131774      1.000000      -0.115715   
PSP_Moneycard   -0.017502   0.017502     -0.115715       1.000000   
PSP_Simplecard   0.063746  -0.063746     -0.149268      -0.254145   
PSP_UK_Card      0.022366  -0.022366     -0.274007      -0.466528   

                PSP_Simplecard  PSP_UK_Card  
success_0             0.063746     0.022366  
success_1            -0.063746    -0.022366  
PSP_Goldcard         -0.149268    -0.274007  
PSP_Moneycard        -0.254145    -0.466528  
PSP_Simplecard        1.000000    -0.601802  
PSP_UK_Card          -0.601802     1.000000  
